In [345]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer

pd.options.display.max_rows = 200
pd.options.display.max_columns = 50

In [346]:
mlb = MultiLabelBinarizer()
df = pd.read_csv('data/combined.csv',index_col='appid').sort_values('appid')

# len(
#     set(mlb.fit(df['tags'].dropna().str.split(', ')).classes_)
#     .difference(set(mlb.fit(df['genres'].dropna().str.split(', ')).classes_))
#     .difference(set(mlb.fit(df['categories'].dropna().str.split(', ')).classes_))
# )
# df['developers'] = df['developers'].str.split(', ')
mlb.fit(df['supported_languages_audio'].dropna().str.split(', ')).classes_

array(['Arabic', 'Bulgarian', 'Czech', 'Danish', 'Dutch', 'English',
       'Finnish', 'French', 'German', 'Greek', 'Hungarian', 'Italian',
       'Japanese', 'Korean', 'Norwegian', 'Polish', 'Portuguese',
       'Portuguese - Brazil', 'Romanian', 'Russian', 'Simplified Chinese',
       'Spanish - Latin America', 'Spanish - Spain', 'Swedish', 'Thai',
       'Traditional Chinese', 'Turkish', 'Ukrainian', 'Vietnamese'],
      dtype=object)

In [347]:
# df = pd.read_csv('data/combined.csv',index_col='appid').sort_values('appid')
# df['developers'] = df['developers'].str.split(', ')
# df['recommendations'].fillna(0,inplace)
# developers_df = pd.DataFrame()
# developers_df['developer'] = mlb.fit(df['developers']).classes_

# def get_developer_owners(developer):
#     mask = df['developers'].apply(lambda x: developer in x)
#     return pd.Series(df['owners'][mask].values,index=df['age_days'][mask]).to_dict()

# def get_developer_hours(developer):
#     mask = df['developers'].apply(lambda x: developer in x)
#     return pd.Series(df['average_forever'][mask].values,index=df['age_days'][mask]).to_dict()

# def get_developer_recommendations(developer):
#     mask = df['developers'].apply(lambda x: developer in x)
#     return pd.Series(df['recommendations'][mask].values,index=df['age_days'][mask]).to_dict()

# developers_df['owners_list'] = developers_df['developer'].apply(get_developer_owners)
# developers_df['hours_list'] = developers_df['developer'].apply(get_developer_hours)
# developers_df['recommendations_list'] = developers_df['developer'].apply(get_developer_recommendations)

# developers_df.set_index('developer', inplace=True)
# developers_df.to_json('data/developers.json')

In [348]:
# df = pd.read_csv('data/combined.csv',index_col='appid').sort_values('appid')
# df['publishers'] = df['publishers'].str.split(', ')
# df.dropna(subset=['publishers'],inplace=True)
# df['recommendations'].fillna(0,inplace=True)


# publishers_df = pd.DataFrame()
# publishers_df['publishers'] = mlb.fit(df['publishers']).classes_

# def get_publisher_owners(publisher):
#     mask = df['publishers'].apply(lambda x: publisher in x)
#     return pd.Series(df['owners'][mask].values,index=df['age_days'][mask]).to_dict()

# def get_publisher_hours(publisher):
#     mask = df['publishers'].apply(lambda x: publisher in x)
#     return pd.Series(df['average_forever'][mask].values,index=df['age_days'][mask]).to_dict()

# def get_publisher_recommendations(publisher):
#     mask = df['publishers'].apply(lambda x: publisher in x)
#     return pd.Series(df['recommendations'][mask].values,index=df['age_days'][mask]).to_dict()

# publishers_df['owners_list'] = publishers_df['publishers'].apply(get_publisher_owners)
# publishers_df['hours_list'] = publishers_df['publishers'].apply(get_publisher_hours)
# publishers_df['recommendations_list'] = publishers_df['publishers'].apply(get_publisher_recommendations)

# publishers_df.set_index('publishers', inplace=True)
# publishers_df.to_json('data/publishers.json')

In [390]:
df = pd.read_csv('data/combined.csv',index_col='appid').sort_values('appid')
df = df.drop_duplicates()

df['ccu_ratio'] = df['ccu']/df['average_2weeks']
df = df[~(df['ccu_ratio'] <= 0.1)]
df = df.drop(columns='ccu_ratio')


df = df[~(df['genres'].str.contains('Utilities') | 
          df['genres'].str.contains('Video Production') | 
          df['genres'].str.contains('Audio Production') |
          df['genres'].str.contains('Animation & Modeling') |
          df['genres'].str.contains('Tutorial') |
          df['genres'].str.contains('Software Training') |
          df['genres'].str.contains('Photo Editing') |
          df['genres'].str.contains('Tutorial') | 
          df['genres'].str.contains('Movie') |
          df['genres'].str.contains('Game Development') |
          df['tags'].str.contains('Utilities') 
         )]

df = df[~(df['name'].str.contains('Idle') | 
          df['name'].str.contains('IDLE')
         )]

# SPLITTING LANGUAGES #############

df['supported_languages'].fillna('',inplace=True)
df['supported_languages_audio'].fillna('',inplace=True)

df['supported_languages'] = df['supported_languages'].str.split(', ')
df['supported_languages_audio'] = df['supported_languages_audio'].str.split(', ')

df['languages_count'] = df['supported_languages'].apply(lambda x: len(x))
df['languages_audio_count'] = df['supported_languages_audio'].apply(lambda x: len(x))


# JOINING LANGUAGES #############

df['supported_languages'] = df['supported_languages'].apply(lambda x: ", ".join(x))
df['supported_languages_audio'] = df['supported_languages_audio'].apply(lambda x: ", ".join(x))


# SPLITTING CATS #############

df.dropna(subset=['categories','genres','tags'],inplace=True)

df['tags'] = df['tags'].str.split(', ')
df['genres'] = df['genres'].str.split(', ')
df['categories'] = df['categories'].str.split(', ')
df['tags'].fillna('',inplace=True)

pruned_tags = (set(mlb.fit(df['tags']).classes_)
                .difference(set(mlb.fit(df['genres']).classes_))
                .difference(set(mlb.fit(df['categories']).classes_)))

df['tags'] = df['tags'].apply(lambda x: [y for y in x if (y in pruned_tags)])



# JOINING CATS #############

df['tags'] = df['tags'].apply(lambda x: ", ".join(x))
df['genres'] = df['genres'].apply(lambda x: ", ".join(x))
df['categories'] = df['categories'].apply(lambda x: ", ".join(x))

# df = df[~(df['tags']=='')]


df['developers'] = df['developers'].str.split(', ')
developers_dict = pd.read_json('data/developers.json').to_dict()


def developer_historical_owners(developers, age):
    global developers_dict
    developer_means = []
    for developer in developers:
        temp = developers_dict.get('owners_list').get(developer)
        developer_means.append(np.mean([temp.get(x) for x in temp.keys() if int(x) > age]))
    return np.mean(developer_means)

def developer_historical_hours(developers, age):
    global developers_dict
    developer_means = []
    for developer in developers:
        temp = developers_dict.get('hours_list').get(developer)
        developer_means.append(np.mean([temp.get(x) for x in temp.keys() if int(x) > age]))
    return np.mean(developer_means)

def developer_historical_recommendations(developers, age):
    global developers_dict
    developer_means = []
    for developer in developers:
        temp = developers_dict.get('recommendations_list').get(developer)
        developer_means.append(np.mean([temp.get(x) for x in temp.keys() if int(x) > age]))
    return np.mean(developer_means)

df['historic_developer_average_owners'] = df[['developers','age_days']].apply(lambda x: developer_historical_owners(x.developers,x.age_days),axis=1).fillna(0)
df['historic_developer_average_hours'] = df[['developers','age_days']].apply(lambda x: developer_historical_hours(x.developers,x.age_days),axis=1).fillna(0)
df['historic_developer_average_recommendations'] = df[['developers','age_days']].apply(lambda x: developer_historical_recommendations(x.developers,x.age_days),axis=1).fillna(0)

publishers_dict = pd.read_json('data/publishers.json').to_dict()
df['publishers'].fillna('',inplace=True)
df['publishers'] = df['publishers'].str.split(', ')

df['top_100_publishers'] = df['publishers'].apply(lambda x: [y for y in x if (y in top_100_publishers)])

df['top_100_publishers'] = df['top_100_publishers'].apply(lambda x: ", ".join(x))

def publisher_historical_owners(publishers, age):
    global publishers_dict
    publisher_means = []
    try:
        for publisher in publishers:
            temp = publishers_dict.get('owners_list').get(publisher)
            publisher_means.append(np.mean([temp.get(x) for x in temp.keys() if int(x) > age]))
    except:
        pass
    return np.mean(publisher_means)

def publisher_historical_hours(publishers, age):
    global publishers_dict
    publisher_means = []
    try:
        for publisher in publishers:
            temp = publishers_dict.get('hours_list').get(publisher)
            publisher_means.append(np.mean([temp.get(x) for x in temp.keys() if int(x) > age]))
    except:
        pass
    return np.mean(publisher_means)

def publisher_historical_recommendations(publishers, age):
    global publishers_dict
    publisher_means = []
    try:
        for publisher in publishers:
            temp = publishers_dict.get('recommendations_list').get(publisher)
            publisher_means.append(np.mean([temp.get(x) for x in temp.keys() if int(x) > age]))
    except:
        pass
    return np.mean(publisher_means)

df['historic_publisher_average_owners'] = df[['publishers','age_days']].apply(lambda x: publisher_historical_owners(x.publishers,x.age_days),axis=1).fillna(0)
df['historic_publisher_average_hours'] = df[['publishers','age_days']].apply(lambda x: publisher_historical_hours(x.publishers,x.age_days),axis=1).fillna(0)
df['historic_publisher_average_recommendations'] = df[['publishers','age_days']].apply(lambda x: publisher_historical_recommendations(x.publishers,x.age_days),axis=1).fillna(0)



# df = df[df['owners']!=20000]
print(len(df))
df['hours_over_age'] = df['average_forever']/df['age_days']
df.to_csv('data/combined_cleaned.csv')

df.sort_values('average_forever',ascending=False).head(50)



/Users/matteofortier/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/matteofortier/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


41647


,name,developers,publishers,platforms,categories,genres,tags,tag_values,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,ccu,is_free,price,initialprice,discount,required_age,controller_support,supported_languages,supported_languages_audio,metacritic,recommendations,release_date,age_days,languages_count,languages_audio_count,historic_developer_average_owners,historic_developer_average_hours,historic_developer_average_recommendations,top_100_publishers,historic_publisher_average_owners,historic_publisher_average_hours,historic_publisher_average_recommendations,hours_over_age
appid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
281750,Munin,[Gojira],[Daedalic Entertainment],"windows, mac, linux","Single-player, Steam Achievements, Full contro...","Adventure, Casual, Indie","Puzzle-Platformer, Puzzle, Platformer, Female ...","67, 66, 65, 54, 53, 51, 29, 25, 25, 16, 14, 13...",641,134,0,500000,293536,0,293722,0,6,0,499.0,499.0,0.0,0,1,"English, French, Italian, German, Spanish - Sp...",,68.0,626.0,2014-06-10,2613,10,1,0.000000e+00,0.000000,0.000000,Daedalic Entertainment,5.733333e+05,273.866667,1854.000000,112.336778
795580,THE DAY Online,[Reloaded Studios],[Reloaded Studios],windows,"Multi-player, Online PvP, Co-op, Online Co-op","Action, Free to Play, Early Access","MOBA, Multiplayer, Team-Based, Online Co-Op, S...","230, 188, 185, 176, 143, 127, 126, 115, 95, 88...",348,251,0,500000,293070,0,293072,0,0,0,0.0,0.0,0.0,0,0,"English, Korean, French, Italian, German, Span...",,NaN,NaN,2018-04-03,1220,18,1,0.000000e+00,0.000000,0.000000,,0.000000e+00,0.000000,0.000000,240.221311
572520,Dropzone,[Sparkypants Studios LLC],[Sparkypants Studios LLC],windows,"Single-player, Multi-player, PvP, Online PvP, ...","Action, Free to Play, Strategy, Early Access","MOBA, Mechs, e-sports, RTS, Sci-fi, Competitiv...","68, 68, 64, 55, 50, 49, 47, 39, 36, 30, 29, 27...",595,263,0,200000,186519,0,186538,0,6,1,0.0,0.0,0.0,0,0,"English, German, French, Italian, Spanish - Sp...",English,NaN,105.0,2017-02-15,1632,10,1,0.000000e+00,0.000000,0.000000,,0.000000e+00,0.000000,0.000000,114.288603
791150,EV3 - Drag Racing,[KABloom Interactive],[KABloom Interactive],"windows, mac","Single-player, Multi-player, PvP, Online PvP, ...","Free to Play, Indie, Racing, Simulation","Automobile Sim, Multiplayer","32, 29, 26, 26, 23, 5",86,88,0,100000,186510,0,186510,0,3,1,0.0,0.0,0.0,0,0,English,,NaN,NaN,2018-02-26,1256,1,1,0.000000e+00,0.000000,0.000000,,0.000000e+00,0.000000,0.000000,148.495223
386480,Blood Code,"[ZiX Solutions, Weixi Studio]",[ZiX Solutions],"windows, mac, linux","Single-player, Steam Achievements, Steam Tradi...","Adventure, Casual, Indie, Simulation, Strategy","Otome, Visual Novel, Anime, Female Protagonist...","139, 73, 59, 53, 42, 41, 37, 37, 33, 33, 19, 1...",1471,526,0,100000,146889,0,220210,0,3,0,999.0,999.0,0.0,0,0,"Simplified Chinese, English, Traditional Chine...",Simplified Chinese,NaN,1696.0,2015-12-28,2047,4,1,0.000000e+00,0.000000,0.000000,,0.000000e+00,0.000000,0.000000,71.758183
377680,Sakura Beach,[Winged Cloud],[Winged Cloud],"windows, linux","Single-player, Steam Trading Cards","Indie, Simulation","Anime, Visual Novel, Mature, Dating Sim, Henta...","102, 87, 71, 60, 60, 55, 52, 34, 20, 15, 12, 1...",1000,258,0,200000,146638,0,219930,0,17,0,999.0,999.0,0.0,0,0,English,,NaN,674.0,2015-08-14,2183,1,1,6.440000e+05,520.200000,1592.200000,Winged Cloud,6.440000e+05,520.200000,1592.200000,67.172698
343630,CroNix,[MAGICS],[MAGICS],windows,"Multi-player, Co-op","Action, Casual, Free to Play, Strategy","Multiplayer, MOBA, Third Person, Online Co-Op,...","126, 95, 86, 66, 61, 55, 52, 44, 34, 33, 27, 2...",212,238,0,500000,139884,0,279762,0,0,1,0.0,0.0,0.0,12,0,"English, French, Italian, German, Spanish - Sp...",English,NaN,NaN,2015-07-17,2211,9,1,0.000000e+00,0.000000,0.000000,,0.000000e+00,0.000000,0.000000,63.267300
407980,Sakura Beach 2,[Winged Cloud],[Winged Cloud],"windows, linux","Single-player, Steam Tra

In [389]:
publisher_keys = list(publishers_dict.get('owners_list').keys())

publisher_count = pd.DataFrame([len(list(publishers_dict.get('owners_list').get(x).values())) for x in publisher_keys],index=publisher_keys)

publisher_count = publisher_count.reset_index().sort_values(0,ascending=False)['index'].head(100)

top_100_publishers = publisher_count.to_list()
top_100_publishers

['Big Fish Games',
 'Plug In Digital',
 'Square Enix',
 'Sekai Project',
 'THQ Nordic',
 'Choice of Games',
 'Ubisoft',
 'Strategy First',
 'Slitherine Ltd.',
 'SEGA',
 'HH-Games',
 'PLAYISM',
 'Devolver Digital',
 'Degica',
 'Laush Studio',
 '8floor',
 '1C Entertainment',
 'Alawar Entertainment',
 'Artifex Mundi',
 'BANDAI NAMCO Entertainment',
 'Electronic Arts',
 'Blender Games',
 'Tero Lunkka',
 'Kagura Games',
 'Ripknot Systems',
 'Paradox Interactive',
 'Daedalic Entertainment',
 'GrabTheGames',
 'tinyBuild',
 '2K',
 'Buka Entertainment',
 'Forever Entertainment S. A.',
 'Microids',
 'RewindApp',
 'Boogygames Studios',
 'MangaGamer',
 'Focus Home Interactive',
 'Back To Basics Gaming',
 'WhisperGames',
 'Gamera Game',
 'KOEI TECMO GAMES CO.',
 'PlayWay S.A.',
 'Sometimes You',
 'Droid Riot',
 'Hosted Games',
 'Nacon',
 'Ghost_RUS Games',
 'HandyGames',
 'Garage Games',
 'XSEED Games',
 'HexWar Games',
 'Dnovel',
 'United Independent Entertainment GmbH',
 'Headup',
 'Iceberg Inter

In [ ]:
df['historic_publisher_average_hours'].describe()

In [ ]:
df = df[df['age_days'] > 14]
df.sort_values('hours_over_age',ascending=False).head(50)


In [ ]:

len(df[df['hours_over_age'] >= 1])

In [ ]:
len(df[df['average_forever']>1000])